In [5]:
import pandas as pd
import numpy as np
random_state=42
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn import metrics
import sklearn
import matplotlib.pyplot as plt
print(sklearn.__version__)

0.23.1


In [6]:
#feature to use
'''
close price 
assetturnover
Total Assets
ROA
total_equity/total_liability
Earnings per Share
Net Operating Cash Flows
EBIT Margin
'''
added_columns=['gdp','unemployment_rate','baa'] # This column contains economic data variables
sample_columns=['prcc_f','at','oancf','revt','epsfi','ebit','lct','lt','seq','ni','ch','stalt','costat','fyear','spcseccd']+added_columns
columns_selected=['prcc_f','at','epsfi','ebit_margin','total_equity/total_liability']+added_columns
preditor=['stalt']

In [7]:
'''
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
def handle_missing(df_healthy,df_bankruptcy):
    columns_to_delete_b=['pll','tie','unwcc','ustdnc','amgw','pvcl','udvp','txtubtxtr','wdp','teq','dt']
    columns_to_delete_h=['unwcc','ustdnc','pvcl','udvp','pll','tie','wdp','amgw','txtubtxtr','teq','dt']
    object_columns=['indfmt', 'consol', 'popsrc', 'datafmt', 'tic', 'curcd', 'costat','fic', 'incorp']
    df_healthy=df_healthy.drop(columns=columns_to_delete_h+object_columns)
    df_bankruptcy=df_bankruptcy.drop(columns=columns_to_delete_b+object_columns)
    imp=IterativeImputer(estimator=RandomForestRegressor(),n_nearest_features=5,verbose=1,random_state=random_state)
    df_all=df_bankruptcy.values
    df=imp.fit_transform(df_all)
    return df_all
    #random regressor imputation
'''
# This method is not good enough

"\nfrom sklearn.experimental import enable_iterative_imputer\nfrom sklearn.impute import IterativeImputer\nfrom sklearn.ensemble import RandomForestRegressor\ndef handle_missing(df_healthy,df_bankruptcy):\n    columns_to_delete_b=['pll','tie','unwcc','ustdnc','amgw','pvcl','udvp','txtubtxtr','wdp','teq','dt']\n    columns_to_delete_h=['unwcc','ustdnc','pvcl','udvp','pll','tie','wdp','amgw','txtubtxtr','teq','dt']\n    object_columns=['indfmt', 'consol', 'popsrc', 'datafmt', 'tic', 'curcd', 'costat','fic', 'incorp']\n    df_healthy=df_healthy.drop(columns=columns_to_delete_h+object_columns)\n    df_bankruptcy=df_bankruptcy.drop(columns=columns_to_delete_b+object_columns)\n    imp=IterativeImputer(estimator=RandomForestRegressor(),n_nearest_features=5,verbose=1,random_state=random_state)\n    df_all=df_bankruptcy.values\n    df=imp.fit_transform(df_all)\n    return df_all\n    #random regressor imputation\n"

In [8]:
def handle_economic():
    # GDP columns
    df_gdp=pd.read_csv('../data/economic_data/gdp.csv',parse_dates=[0],names=['year','gdp'],header=0)
    df_gdp['gdp']=df_gdp['gdp'].pct_change()
    df_gdp['year']=df_gdp['year'].apply(lambda x: x.year)
    
    #unemployment columns
    df_un=pd.read_csv('../data/economic_data/unemployment_rate.csv',parse_dates=[0],names=['year1','unemployment_rate'],header=0)
    df_un['unemployment_rate']=df_un['unemployment_rate']/100
    df_un['year1']=df_un['year1'].apply(lambda x: x.year)
    
    #bbb spread
    df_baa=pd.read_csv('../data/economic_data/baa_yield.csv',parse_dates=[0],names=['year2','baa'],header=0)
    df_baa['baa']=df_baa['baa']/100
    df_baa['year2']=df_baa['year2'].apply(lambda x:x.year)
    df_baa=df_baa.groupby('year2').mean().reset_index(level=0) # 12 month baa yield average
    
    # merge two datasheet
    df=df_gdp.merge(df_un,how='left',left_on='year',right_on='year1').drop(columns='year1')
    df=df.merge(df_baa,how='left',left_on='year',right_on='year2').drop(columns='year2')
    return df.dropna().copy()

In [7]:
def sample_data(freq=0.01,columns=sample_columns):
    #filename:healthy or bankruptcy
    df1=pd.read_csv('../data/accouting_data/healthy_accouting_stalt.csv')
    df2=pd.read_csv('../data/accouting_data/bankruptcy_accouting_stalt.csv')
    df2['stalt']=1.0 #label bankruptcy
    df1['stalt']=0.0 # label healthy
    
     # fill original sector in order to sample
    df1['spcseccd'].fillna(0,inplace=True) 
    df2['spcseccd'].fillna(0,inplace=True)
    
    
    #merge data
    df=handle_economic()
    df1=df1.merge(df,how='left',left_on='fyear',right_on='year').drop(columns='year')
    df2=df2.merge(df,how='left',left_on='fyear',right_on='year').drop(columns='year')
    
    # naive ways of handling nan data(reasonable)
    df1=df1[df1['costat']=='A'] #get all active companies data
    df1=df1[columns].dropna()
    df2=df2[columns].dropna()
    
    
    sample_df=df1.groupby('spcseccd').apply(lambda x: x.sample(frac=freq,random_state=random_state))
    # drop spcseccd columns in order to reset index
    sample_df.drop(columns='spcseccd',inplace=True)
    sample_df.reset_index(level=0,inplace=True)
    #statified sample
    #cols = sample_df.columns.tolist()
    #cols=cols[-1:] + cols[:-1]
    sample_df=sample_df[columns]
    df_final=sample_df.append(df2).reset_index().drop(columns='index')
    
    # compute ratio
    #1. ebit margin
    df_final['ebit_margin']=df_final['ebit']/df_final['revt']
    df_final.drop(columns='ebit',inplace=True)
    
    #2. ROA
    #df_final['roa']=df_final['ni']/df_final['at']
    #df_final.drop(columns='ni',inplace=True)
    
    #3.quick ratio
    #df_final['quick_ratio']=df_final['ch']/df_final['lct']
    #df_final.drop(columns=['ch','lct'],inplace=True)
    
    #4. asset turnover
    #df_final['asset_turnover']=df_final['revt']/df_final['at']
    #df_final.drop(columns='revt',inplace=True)
    
    # 5. cash flow return
    #df_final['cash_flow_to_asset']=df_final['oancf']/df_final['at']
    #df_final.drop(columns='oancf',inplace=True)
    
    #6. total equity/total liablity
    df_final['total_equity/total_liability']=df_final['seq']/df_final['lt']
    
    #5. get log on asset
    df_final['at']=np.log(df_final['at'])
    df_final['prcc_f']=np.log(df_final['prcc_f'])
    
    #drop inf value
    df_final=df_final.replace([np.inf, -np.inf], np.nan)
    
    return df_final.dropna().copy()
        
        

In [15]:
def make_test_train(df,columns=columns_selected,pred=preditor,random_state=random_state):
    #handle with nan data
    X=df[columns_selected]
    y=df[pred]
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=random_state,stratify=y)
    
    return X_train,X_test,y_train.values.flatten(),y_test.values.flatten()
    
    #select feature for ML training

In [1]:
def calc_metrics(model,X_test,y_test,threshold=0.5,type_=None):
    """
    Arguments:
    Model: - trained model such as Logistic Regression
    X_test - np.array of test set
    y_test -binary label in range {0,1} 
    """
    if model is None:
        return 0.0,0.0,0.0
    predicted_prob=np.zeros(X_test.shape[0])
    predicted_binary=np.zeros(X_test.shape[0])
    if type_==1:
        #Logistic Regression using sklearn
        predicted_prob=model.predict_proba(X_test)[:,1]
        predicted_binary=(predicted_prob>threshold).astype(int)
    
    if type_==0:
        predicted_prob=model.predict(X_test)
        predicted_binary=(predicted_prob>threshold).astype(int)
    
    fpr,tpr,_=metrics.roc_curve(y_test,predicted_prob,pos_label=1)
    
    #compute AUC from prediction score
    roc_auc=metrics.auc(fpr,tpr)
    ks=np.max(tpr-fpr)
    
    #accuracy score
    accuracy_score=metrics.accuracy_score(y_test,predicted_binary)
        
    try:
        plt.title('Logistic Regression ROC Curve')
        plt.plot(fpr,tpr,'b',label='AUC=%0.2f' % roc_auc)
        plt.legend(loc='lower right')
        plt.plot([0,1],[0,1],'r--')
        plt.xlabel('False Positive rate')
        plt.ylabel('True Positive rate')
        plt.savefig('../picture/ROC_curve_1.png')
        plt.show()
    except:
        pass
    
    return roc_auc,accuracy_score,ks